In [1]:
import os
import glob # Adds wildcard-support for file exploration
import numpy as np
%matplotlib
import matplotlib.pyplot as plt

# Load functions from file "utils.py"
import utils

Using matplotlib backend: Qt5Agg


### Load bizeps dataset

In [2]:
data_path = os.path.join("..", "data", "crunches", "raw")
gyro_data_files = glob.glob(os.path.join(data_path,"*_gyro_sensor_data_*.txt"))
accel_data_files = glob.glob(os.path.join(data_path,"*_accel_sensor_data_*.txt"))

print(*gyro_data_files, sep="\n")
print("\n")
print(*accel_data_files, sep="\n")

../data/crunches/Crunches_gyro_sensor_data_1617807292329.txt
../data/crunches/Crunches_gyro_sensor_data_1617807414485.txt
../data/crunches/Crunches_gyro_sensor_data_1617807602040.txt
../data/crunches/Crunches_gyro_sensor_data_1617807753937.txt


../data/crunches/Crunches_accel_sensor_data_1617807292329.txt
../data/crunches/Crunches_accel_sensor_data_1617807414485.txt
../data/crunches/Crunches_accel_sensor_data_1617807602040.txt
../data/crunches/Crunches_accel_sensor_data_1617807753937.txt


In [3]:
file_idx = 0

gyro_data_file = gyro_data_files[file_idx]
accel_data_file = accel_data_files[file_idx]
gyro_data = np.loadtxt(gyro_data_file, delimiter=";")
accel_data = np.loadtxt(accel_data_file, delimiter=";")

gyro_data[:,0] = gyro_data[:,0] * 1e-3 # Convert from milli-seconds to seconds
accel_data[:,0] = accel_data[:,0] * 1e-3 # Convert from milli-seconds to seconds

### Interpolate and resample
The data from the sensors is not captured at a uniform sampling rate. Using linear interpolation, a sampling-rate of $100\ \text{Hz}$ is applied.

In [4]:
t_gyro_new, gyro_data_new = utils.resample_3d_timeseries(t=gyro_data[:,0], y=gyro_data[:,1:], f_resample=100.0)
t_accel_new , accel_data_new = utils.resample_3d_timeseries(t=accel_data[:,0], y=accel_data[:,1:], f_resample=100.0)

# Crop both data arrays to the same length.
if t_gyro_new[-1] < t_accel_new[-1]:
    T_max = t_gyro_new[-1]
    idx = t_accel_new <= T_max
    t_accel_new = t_accel_new[idx]
    accel_data_new = accel_data_new[idx,:]
else:
    T_max = t_accel_new[-1]
    idx = t_gyro_new <= T_max
    t_gyro_new = t_gyro_new[idx]
    gyro_data_new = gyro_data_new[idx,:]

# Check both lengths are now really the same.
assert len(t_gyro_new) == len(t_accel_new)

In [5]:
T_clip_start = 6
T_clip_end = 33

### Clipping Gyro-data

In [6]:
fig_gyro = utils.plot_sensor_data(np.column_stack((t_gyro_new, gyro_data_new)), title="Gyro", ylabels=("x", "y", "z"))

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


In [7]:
idx_clip_start = np.where(t_gyro_new >= T_clip_start)[0][0]
idx_clip_end = np.where(t_gyro_new <= T_clip_end)[0][-1]

In [8]:
t_gyro_new_clip = t_gyro_new[idx_clip_start:idx_clip_end+1] - t_gyro_new[idx_clip_start]
gyro_data_new_clip = gyro_data_new[idx_clip_start:idx_clip_end+1]

In [9]:
data_total = np.column_stack((t_gyro_new_clip, gyro_data_new_clip))
data_total.shape

(2701, 4)

In [10]:
_ = utils.plot_sensor_data(np.column_stack((t_gyro_new_clip, gyro_data_new_clip)), title="Gyro clipped", ylabels=("x", "y", "z"), fig=fig_gyro)

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


In [11]:
data_file_basename = os.path.basename(gyro_data_file)[:-4]
clipped_data_filename = os.path.join(data_path, data_file_basename + "_clipped.npy")
clipped_data_filename

'../data/crunches/Crunches_gyro_sensor_data_1617807292329_clipped.npy'

In [12]:
#np.save(clipped_data_filename, data_total, allow_pickle=True)

### Clipping Accelerometer-data

In [13]:
fig_accel = utils.plot_sensor_data(np.column_stack((t_accel_new, accel_data_new)), title="Accelerometer", ylabels=("x", "y", "z"))

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


In [14]:
idx_clip_start = np.where(t_accel_new >= T_clip_start)[0][0]
idx_clip_end = np.where(t_accel_new <= T_clip_end)[0][-1]

In [15]:
t_accel_new_clip = t_accel_new[idx_clip_start:idx_clip_end+1] - t_accel_new[idx_clip_start]
accel_data_new_clip = accel_data_new[idx_clip_start:idx_clip_end+1]

In [16]:
data_total = np.column_stack((t_accel_new_clip, accel_data_new_clip))
data_total.shape
fig_accel = utils.plot_sensor_data(np.column_stack((t_accel_new_clip, accel_data_new_clip)), title="Accelerometer", ylabels=("x", "y", "z"), fig=fig_accel)

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


In [17]:
data_file_basename = os.path.basename(accel_data_file)[:-4]
clipped_data_filename = os.path.join(data_path, data_file_basename + "_clipped.npy")
clipped_data_filename

'../data/crunches/Crunches_accel_sensor_data_1617807292329_clipped.npy'

In [18]:
#np.save(clipped_data_filename, data_total, allow_pickle=True)